In [98]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [99]:
#Create empty lists to store the data we are about to collect from the website
#We would like to collect the names of the cars, the mileage, the price, the rating, the number of reviews the car received

car_name = []
car_mileage = []
car_price = []
car_rating = []
car_ratingcount = []   

In [100]:
#We need to create a variable to stor the url of the website, I'm calling this variable website_url. We then need to send a request to the website
#and scrape multiple pages

for i in range(1,6):
    website_url ="https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=nissan&maximum_distance=20&mileage_max=&models[]=nissan-rogue&monthly_payment=&page=" + str(i) + "&page_size=20&sort=best_match_desc&stock_type=used&trims[]=nissan-rogue-sv&year_max=&year_min=&zip=02215"
    
    #make request to the website
    web_request = requests.get(website_url)
    
    # Create an object to store the html elements of the request
    soup = BeautifulSoup(web_request.content,'html.parser')
    
    # Create a list that contains the results of the web page
    results_list = soup.find_all('div', {'class':'vehicle-card'})
    
    for i in results_list:
    
        #get the names of the cars. If the name of the car is not present, append 'n/a' to the list
        try:
            car_name.append(i.find('h2').get_text())
        except:
            car_name.append('n/a')

        #get the mileage of the cars
        try:
            car_mileage.append(i.find('div', {'class' : 'mileage'}).get_text())
        except:
            car_mileage.append('n/a')

        #get the price of the car
        try:
            car_price.append(i.find('span', {'class' : 'primary-price'}).get_text())
        except:
            car_price.append('n/a')

        #get the rating
        try:
            car_rating.append(i.find('span', {'class': 'sds-rating__count'}).get_text())
        except:
            car_rating.append('n/a')

        #get the number of reviews
        try:
            car_ratingcount.append(i.find('span', {'class': 'sds-rating__link'}).get_text())
        except:
            car_ratingcount.append('n/a')

In [101]:
# Create a DataFrame

car_info = pd.DataFrame({'Name': car_name, 'Mileage': car_mileage, 'Price': car_price, 'Rating': car_rating, 'Reviews': car_ratingcount})

In [102]:
#print the DataFrame
car_info

,Name,Mileage,Price,Rating,Reviews
0,2022 Nissan Rogue SV,"20,289 mi.","$29,726",n/a,(4 reviews)
1,2021 Nissan Rogue SV,"26,668 mi.","$28,444",4.7,"(1,504 reviews)"
2,2020 Nissan Rogue SV,"45,636 mi.","$23,994",4.7,"(2,556 reviews)"
3,2020 Nissan Rogue SV,"22,326 mi.","$24,949",4.7,"(2,556 reviews)"
4,2021 Nissan Rogue SV,"51,979 mi.","$25,564",n/a,(92 reviews)
...,...,...,...,...,...
97,2020 Nissan Rogue SV,"47,582 mi.","$26,434",n/a,n/a
98,2021 Nissan Rogue SV,"50,402 mi.","$27,564",n/a,n/a
99,2020 Nissan Rogue SV,"15,711 mi.","$27,499",n/a,n/a
100,2021 Nissan Rogue SV,"23,445 mi.","$31,490",n/a,n/a


In [103]:
#Let's clean the data frame

#The first thing we should do is to edit the Reviews column

car_info['Reviews'] = car_info['Reviews'].apply(lambda x:x.strip('reviews)').strip('(')) #remove the word reviews and the parentheses
car_info['Rating'] = car_info['Rating'].apply(lambda x:x.replace('n/a', '0')) #remove 'n/a'
car_info['Reviews'] = car_info['Reviews'].apply(lambda x:x.replace('n/a', '0')) 
car_info['Mileage'] = car_info['Mileage'].apply(lambda x:x.replace('mi.', '')) #remove 'mi.' from the mileage column
car_info['Price'] = car_info['Price'].apply(lambda x:x.replace('$', '')) # remove the $ from the Price column
car_info['Price'] = car_info['Price'].apply(lambda x:x.replace(',', '')) #remove the ',' from the Price column
car_info['Mileage'] = car_info['Mileage'].apply(lambda x:x.replace(',', '')) #remove the ',' from the Mileage column
car_info['Reviews'] = car_info['Reviews'].apply(lambda x:x.replace(',', '')) #remove the ',' from the Reviews column
car_info['Price'] = car_info['Price'].apply(lambda x:x.replace('Not Priced', '0')) #there's one car where the value is 'not priced', this should be converted to a number

In [104]:
car_info

,Name,Mileage,Price,Rating,Reviews
0,2022 Nissan Rogue SV,20289,29726,0,4
1,2021 Nissan Rogue SV,26668,28444,4.7,1504
2,2020 Nissan Rogue SV,45636,23994,4.7,2556
3,2020 Nissan Rogue SV,22326,24949,4.7,2556
4,2021 Nissan Rogue SV,51979,25564,0,92
...,...,...,...,...,...
97,2020 Nissan Rogue SV,47582,26434,0,0
98,2021 Nissan Rogue SV,50402,27564,0,0
99,2020 Nissan Rogue SV,15711,27499,0,0
100,2021 Nissan Rogue SV,23445,31490,0,0


In [105]:
#The Name column has a lot of important data on the Year, Company, Vehicle make and model

car_info[["Year", "Company", "Make", "Model"]] = car_info["Name"].str.split(" ", expand=True)

In [91]:
car_info

,Company,Make,Model,Year,Price,Mileage,Rating,Reviews
0,Nissan,Rogue,SV,2022,29726,20289,0,4
1,Nissan,Rogue,SV,2019,23499,33514,4.7,665
2,Nissan,Rogue,SV,2021,28444,26668,4.7,1504
3,Nissan,Rogue,SV,2020,23994,45636,4.7,2556
4,Nissan,Rogue,SV,2020,24949,22326,4.7,2556
...,...,...,...,...,...,...,...,...
97,Nissan,Rogue,SV,2020,26434,47582,0,0
98,Nissan,Rogue,SV,2021,27564,50402,0,0
99,Nissan,Rogue,SV,2020,27499,15711,0,0
100,Nissan,Rogue,SV,2021,31490,23445,0,0


In [106]:
# I would like to chang ethe position of the columns
car_info = car_info.iloc[:,[0,6,7,8,5,2,1,3,4]]

In [107]:
car_info

,Name,Company,Make,Model,Year,Price,Mileage,Rating,Reviews
0,2022 Nissan Rogue SV,Nissan,Rogue,SV,2022,29726,20289,0,4
1,2021 Nissan Rogue SV,Nissan,Rogue,SV,2021,28444,26668,4.7,1504
2,2020 Nissan Rogue SV,Nissan,Rogue,SV,2020,23994,45636,4.7,2556
3,2020 Nissan Rogue SV,Nissan,Rogue,SV,2020,24949,22326,4.7,2556
4,2021 Nissan Rogue SV,Nissan,Rogue,SV,2021,25564,51979,0,92
...,...,...,...,...,...,...,...,...,...
97,2020 Nissan Rogue SV,Nissan,Rogue,SV,2020,26434,47582,0,0
98,2021 Nissan Rogue SV,Nissan,Rogue,SV,2021,27564,50402,0,0
99,2020 Nissan Rogue SV,Nissan,Rogue,SV,2020,27499,15711,0,0
100,2021 Nissan Rogue SV,Nissan,Rogue,SV,2021,31490,23445,0,0


In [108]:
# Drop the column Name and its no longer needed
car_info = car_info.drop(["Name"], axis = 1)

In [109]:
car_info

,Company,Make,Model,Year,Price,Mileage,Rating,Reviews
0,Nissan,Rogue,SV,2022,29726,20289,0,4
1,Nissan,Rogue,SV,2021,28444,26668,4.7,1504
2,Nissan,Rogue,SV,2020,23994,45636,4.7,2556
3,Nissan,Rogue,SV,2020,24949,22326,4.7,2556
4,Nissan,Rogue,SV,2021,25564,51979,0,92
...,...,...,...,...,...,...,...,...
97,Nissan,Rogue,SV,2020,26434,47582,0,0
98,Nissan,Rogue,SV,2021,27564,50402,0,0
99,Nissan,Rogue,SV,2020,27499,15711,0,0
100,Nissan,Rogue,SV,2021,31490,23445,0,0


In [56]:
#You can export the DataFrame to excel if you choose to proceed with excel
car_info.to_excel('multiple_pages.xlsx', index = False)

In [ ]:
#You can export the DataFrame to a csv file if needed
car_info.to_csv('multiple_pages.csv', index = False)

In [87]:
#Let's determine the datatypes for each column within our car_info DataFrame
car_info.dtypes

Company    object
Make       object
Model      object
Year       object
Price      object
Mileage    object
Rating     object
Reviews    object
dtype: object

In [92]:
#All of the columns are listed as objects. We need to change the datatypes for at least some of these columns

#Year should be converted to int
car_info["Year"] = car_info["Year"].astype(int)

In [93]:
car_info["Price"] = car_info["Price"].astype(float)

In [94]:
car_info["Mileage"] = car_info["Mileage"].astype(float)

In [95]:
car_info["Rating"] = car_info["Rating"].astype(float)

In [96]:
car_info["Reviews"] = car_info["Reviews"].astype(int)

In [97]:
car_info.dtypes

Company     object
Make        object
Model       object
Year         int32
Price      float64
Mileage    float64
Rating     float64
Reviews      int32
dtype: object

In [73]:
car_info

,Company,Make,Model,Year,Price,Mileage,Rating,Reviews
0,Nissan,Rogue,SV,2022,29726,20289,0.0,0
1,Nissan,Rogue,SV,2019,23499,33514,4.7,642
2,Nissan,Rogue,SV,2021,28444,26668,4.7,1502
3,Nissan,Rogue,SV,2018,20998,71543,4.7,1502
4,Nissan,Rogue,SV,2021,25564,51979,0.0,92
...,...,...,...,...,...,...,...,...
95,Nissan,Rogue,SV,2020,26434,47582,0.0,0
96,Nissan,Rogue,SV,2021,27564,50402,0.0,0
97,Nissan,Rogue,SV,2020,27499,15711,0.0,0
98,Nissan,Rogue,SV,2021,31490,23445,0.0,0


In [ ]:
#create a new column called Age (Current year - Year)

In [ ]:
#scatter plot & Other visualizations

In [ ]:
#depreciation - regression